# Importando Bibliotecas

In [1]:
import pandas as pd
import re
import plotly_express as px

# Abrindo o Arquivo

In [2]:
anvisa_df = pd.read_excel(r'C:\temp\Python Projects\anvisa\csv_files\consulta_ensaios.xls') # lembrar de adicionar o parâmetro opcional skipnrows=4 caso seja o arquivo filtrado

# Funções Personalizadas

In [3]:
def new_columns(row: pd.Series) -> pd.Series:
    pattern = r'^(.*?)\s*\((.*?)\s*-\s*(\d+)\)$'  # Regex para separar os campos
    
    if pd.notna(row['Instituição de Pesquisa/Investigador/Número de Pacientes']):
        match = re.match(pattern, row['Instituição de Pesquisa/Investigador/Número de Pacientes'])
        
        if match is not None:
            row['Instituição de Pesquisa'] = match.group(1)  # Nome da instituição
            row['Investigador'] = match.group(2)             # Nome do investigador
            row['Número de Pacientes'] = match.group(3)      # Número de pacientes
        else:
            row['Instituição de Pesquisa'] = 'Não há dados'
            row['Investigador'] = 'Não há dados'             
            row['Número de Pacientes'] = 'Não há dados'
    else:
        row['Instituição de Pesquisa'] = 'Não há dados'
        row['Investigador'] = 'Não há dados'             
        row['Número de Pacientes'] = 'Não há dados'
    
    return row

def new_columns_2(row: pd.Series) -> pd.Series:
    pattern = r'^(.*?)\s-\s(.*?)$'

    if pd.notna(row['Doença / CID10']):
        match = re.match(pattern, row['Doença / CID10'])

        if match != 0:
            row['Doença'] = match.group(2)
            row['CID10'] = match.group(1)
        else:
            row['Doença'] = '-'
            row['CID10'] = '-'
    return row

# Iniciando o Processo de Depuração

In [4]:
# num_processo = anvisa_df['Número do Processo'].unique()
# # duplicata_num_processo = anvisa_df['Número do Processo'].duplicated()
# # teste = anvisa_df[duplicata_num_processo]

anvisa_df_filtered = anvisa_df.drop_duplicates(subset=['Número do Processo'])

## Segmentando Instituição de Pesquisa/Investigador/Número de Pacientes em 3

In [5]:
splited_df = anvisa_df_filtered.copy()
splited_df["Instituição de Pesquisa/Investigador/Número de Pacientes"] = anvisa_df_filtered["Instituição de Pesquisa/Investigador/Número de Pacientes"].str.split(';')

# exploded_df = splited_df.copy().reset_index(drop=True)
exploded_df= splited_df.explode('Instituição de Pesquisa/Investigador/Número de Pacientes')

### Utilizando Regex Para Coletar os Campos

In [6]:
# Aplicando a função em cada linha do DataFrame
exploded_df = exploded_df.apply(new_columns, axis=1)

## Utilizando Regex para Segmentar Doença/CID 10 em 2

In [7]:
exploded_df = exploded_df.apply(new_columns_2, axis=1)

# Criando a Tabela Final

In [8]:
final_df = exploded_df[['Patrocinador do Estudo', 'Número do Processo', 'Nome ou Código do Medicamento Experimental', 'Classe Terapêutica', 'Tipo de Medicamento Experimental', 'Doença', 'CID10', 'Nome do Protocolo Clínico', 'Título do Estudo', 'Fase do Estudo', 'Tipo de Estudo', 'Situação do Estudo', 'Instituição de Pesquisa', 'Investigador', 'Número de Pacientes']].copy()
final_df = final_df.fillna('Não há dados')
final_df.to_csv('anvisa_filtered.csv', encoding='utf8', index=False)

In [ ]:
other_ids = pd.read_csv('ct_anvisa.csv')
other_ids = other_ids.rename(columns={'Other IDs': 'Nome do Protocolo Clínico'})

In [ ]:
teste = pd.merge(final_df, other_ids, on='Nome do Protocolo Clínico')

# Criando visualizações

In [9]:
pie = final_df.groupby('Situação do Estudo')['Número do Processo'].nunique().reset_index()

piechart = px.pie(
    pie,
    values='Número do Processo',
    names='Situação do Estudo',
    title=r'% da Situações dos Estudos'
)

piechart.show()

In [10]:
v_bar = final_df.groupby('Patrocinador do Estudo')['Número do Processo'].size().reset_index().sort_values(by='Número do Processo', ascending=False)
v_bar = v_bar.head(40)
vertical_bar = px.bar(
    v_bar,
    orientation='h',
    x='Número do Processo',
    y='Patrocinador do Estudo',
    color='Patrocinador do Estudo',
    title='Total por Patrocinador'
)

vertical_bar.update_layout(showlegend=False)
# vertical_bar.update_yaxes(visible=False)

vertical_bar.show()

In [11]:
pie_drug = final_df.groupby('Tipo de Medicamento Experimental')['Número do Processo'].size().reset_index().sort_values(by='Número do Processo', ascending=False)

pie_drug_chart = px.bar(
    pie_drug,
    orientation='h',
    x='Número do Processo',
    y='Tipo de Medicamento Experimental',
    color='Tipo de Medicamento Experimental',
    title='Total por Tipo de Medicamento Experimental'
)

pie_drug_chart.show()

In [12]:
bar_class = final_df.groupby('Classe Terapêutica')['Número do Processo'].size().reset_index().sort_values(by='Número do Processo', ascending=False)
bar_class = bar_class.head(5)

bar_class_chart = px.bar(
    bar_class,
    orientation='h',
    x='Número do Processo',
    y='Classe Terapêutica',
    color='Classe Terapêutica',
    title='Total por Classe Terapêutica'
)

bar_class_chart.show()

In [13]:
pie_estudo = final_df.groupby("Tipo de Estudo")["Número do Processo"].nunique().reset_index()

pie_estudo_chart = px.pie(
    pie_estudo,
    names="Tipo de Estudo",
    values="Número do Processo",
    title='% Tipo de Estudo'
)

pie_estudo_chart.show()

In [25]:
dado_em_casa = pd.read_csv(r"C:\Users\Bruno Gomes\Downloads\DADOS_ABERTOS_MEDICAMENTOS.csv", encoding='latin1', delimiter=';')
laele = pd.read_csv(r"C:\Users\Bruno Gomes\Downloads\TA_PRODUTO_SAUDE_SITE.csv", encoding='latin1', delimiter=';')

In [27]:
dado_em_casa = dado_em_casa.rename(columns={'NUMERO_PROCESSO': 'Número do Processo'})

In [28]:
novo_sla = pd.merge(dado_em_casa, final_df, on='Número do Processo')
